In [2]:
#imports
import pandas as pd
import numpy as np
import tensorflow as tf

from Functions.conn import db_upload
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import load_model
from sklearn.preprocessing import minmax_scale
print(tf.__version__)


2.0.0-alpha0


In [3]:
verbosity = 1
groupSize = 288

index_names = [ 'Date', 'ASA51', 'XNDX', 'SPXT', 'CACR', 'DAX', 'NKYTR', 'HSI1', 'KSP2TR']

subset = [ 'ASA51', 'XNDX', 'SPXT', 'CACR', 'DAX', 'NKYTR', 'HSI1', 'KSP2TR' ]

#note, raw.csv omits 'date' label in cell A1 of csv but incl's all index names in A2++ so csv can be read more easily but re "dropna"
#rule below, the header row is omitted from all program calculations
index_data = pd.read_csv( 'raw.csv', names = index_names ) # read file, insert header row
index_data.dropna( inplace = True ) # this removes rows with >=1 #na, being 234/1,304 data points in v1 dataset, inplace= True, 'do operation inplace and return None.''
# index_data.describe() #prints some table metrics, not part of any calculation hereafter.
print("index_data shape is ")
print(index_data.shape)
index_data.to_csv('out.csv', index=False)


ASAMin=0
ASAMax=0
ASArange=0

#function to create trainingArray that groups data into lists of 'groupSize' date rows (atm 288) * all indices' data (8)
#creating a master list of 780 rows of 2304 columns, essentially 780 moving windows of all data
def groupData( data, groupSize ):
	trainingArray = []
	for i in range( data.size ):
		if i >= groupSize and i < data.shape[0] :
			trainingListSingleEntry = []
			for ii in range( groupSize ):
				trainingListSingleEntry = np.concatenate( ( trainingListSingleEntry, data[ i - ii -1] ), axis=None )
			trainingArray.append( trainingListSingleEntry )
	return trainingArray


def groupLabels( data, groupSize ):
	trainingArray = []
	for i in range( data.size):
		if i >= groupSize:
			trainingListSingleEntry = data[i]
			trainingArray.append( trainingListSingleEntry )
	return trainingArray

def doTraining():
	trainingData = index_data[ subset ].values
	trainingData = minmax_scale( trainingData )
	trainingData = groupData( trainingData, groupSize = groupSize )
	trainingData = np.array( trainingData )
# 	print("training data 1", trainingData[0])
# 	print(trainingData.shape)

#trainingLabels being target, output dataset
	trainingLabels = index_data[ 'ASA51' ].values
	trainingLabels = minmax_scale( trainingLabels )
# 	print("training label size is ")
# 	print(trainingLabels.size)
	trainingLabels = groupLabels( trainingLabels, groupSize = groupSize )
	trainingLabels = np.array( trainingLabels )
	print("training label row 1",trainingLabels[0])

# 	print("training label shape is ")
# 	print(trainingLabels.shape)

	model = Sequential()
	print("input layer set to:", len( trainingData[0] ))
	# going into a 32-unit layer
	model.add( Dense( 32, input_dim = len( trainingData[0] ), kernel_initializer = 'normal', activation = 'relu' ) )
	# Another hidden layer of 16 units
	model.add( Dense( 32, kernel_initializer = 'normal', activation = 'relu' ) )
	# Another hidden layer of 16 units
	model.add( Dense( 32, kernel_initializer = 'normal', activation = 'relu' ) )
	# Output layer with a binary classification ( Democrat or Republican )
	model.add( Dense( 1 ) )

	# Compile model
	model.compile( loss = 'mse', optimizer = 'rmsprop', metrics = [ 'mae' ] )

	# Train model
	model.fit( trainingData, trainingLabels, epochs = 100, batch_size = 50, verbose = verbosity )

	# Grade the model
	scores = model.evaluate( trainingData, trainingLabels, verbose = verbosity )
	print( "%s: %.2f%%" % ( model.metrics_names[1], scores[1]*100 ) )

	# Save the model
	model.save( 'BHMarket_Model.h5' )
	# plot_model(model, to_file='modelplotbh.png')
    
def doPrediction():
	trainingData = index_data[ subset ].values
	ASAmin =  float((trainingData.min(axis=0))[0])
	print("min in trainingData is: ", ASAmin)
	ASAmax =  float((trainingData.max(axis=0))[0])
	print("max in trainingData is: ", ASAmax)
	ASArange =  ASAmax-ASAmin
	trainingData = minmax_scale( trainingData )
	trainingData = groupData( trainingData, groupSize = groupSize )
	inputData = trainingData[-2]
	trainingLabels = index_data[ 'Date' ].values
	trainingLabels = groupLabels( trainingLabels, groupSize = groupSize )
	date = trainingLabels[-1]
# 	print( "training data shape is", inputData.shape )

	loaded_model = load_model( 'BHMarket_Model.h5' )

	# loaded_model = load_model( 'LSMarket_Model.h5' )
	# evaluate loaded model on test data
	loaded_model.compile( loss = 'mse', optimizer = 'rmsprop', metrics = [ 'mae' ] )
	# Predict things...
# 	print( inputData )
# 	print( inputData.shape )
	prediction_1 = loaded_model.predict( inputData.reshape( (1, 2304) ), batch_size = None, verbose = verbosity, steps = None )
	print("+1 day prediction scaled is:", prediction_1)    
	prediction_1=prediction_1*ASArange+ASAmin
	print("+1 day prediction is:", prediction_1)

doTraining()

print("prediction is: ")
doPrediction()

index_data shape is 
(1070, 9)
training label row 1 0.29212832891014195
input layer set to: 2304
Epoch 1/100
782/782 [==============================] - 1s 1ms/sample - loss: 0.0583 - mae: 0.1709
Epoch 2/100
782/782 [==============================] - 0s 61us/sample - loss: 0.0198 - mae: 0.1149
Epoch 3/100
782/782 [==============================] - 0s 70us/sample - loss: 0.0192 - mae: 0.1110
Epoch 4/100
782/782 [==============================] - 0s 89us/sample - loss: 0.0193 - mae: 0.1091
Epoch 5/100
782/782 [==============================] - 0s 70us/sample - loss: 0.0110 - mae: 0.0822
Epoch 6/100
782/782 [==============================] - 0s 83us/sample - loss: 0.0145 - mae: 0.0960
Epoch 7/100
782/782 [==============================] - 0s 103us/sample - loss: 0.0120 - mae: 0.0883
Epoch 8/100
782/782 [==============================] - ETA: 0s - loss: 0.0113 - mae: 0.083 - 0s 103us/sample - loss: 0.0112 - mae: 0.0836
Epoch 9/100
782/782 [==============================] - 0s 84us/sample - 

782/782 [==============================] - 0s 88us/sample - loss: 0.0055 - mae: 0.0484
Epoch 82/100
782/782 [==============================] - 0s 89us/sample - loss: 0.0025 - mae: 0.0384
Epoch 83/100
782/782 [==============================] - 0s 85us/sample - loss: 0.0034 - mae: 0.0417
Epoch 84/100
782/782 [==============================] - 0s 107us/sample - loss: 0.0028 - mae: 0.0399
Epoch 85/100
782/782 [==============================] - 0s 96us/sample - loss: 0.0024 - mae: 0.0368
Epoch 86/100
782/782 [==============================] - 0s 98us/sample - loss: 0.0033 - mae: 0.0442
Epoch 87/100
782/782 [==============================] - 0s 75us/sample - loss: 0.0025 - mae: 0.0401
Epoch 88/100
782/782 [==============================] - 0s 65us/sample - loss: 0.0030 - mae: 0.0414
Epoch 89/100
782/782 [==============================] - 0s 65us/sample - loss: 0.0030 - mae: 0.0412
Epoch 90/100
782/782 [==============================] - 0s 64us/sample - loss: 0.0090 - mae: 0.0695
Epoch 91/100

In [ ]:
# import mysql.connector
# mydb = mysql.connector.connect(
#   host="localhost",
#   user="root",
#   passwd="",
#   database="a3d"
# )

# def db_upload(sql, val):
#   mycursor = mydb.cursor()
#   mycursor.execute(sql, val)
#   mydb.commit()
#   print(mycursor.rowcount, "record inserted.")


# sql = "INSERT INTO daily_ai_indicators (metric, value) VALUES (%s, %s)"
# val = ("delta_strategy", 123477)
# db_upload(sql,val)